## Add City

In [11]:
import pandas as pd
import numpy as np
import gc
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
import lightgbm as lgb
# from catboost import CatBoostRegressor
from tqdm import tqdm_notebook
from itertools import product

In [12]:
import os
input_folder = "../predict_future_sales" # or input_folder = os.getcwd()
os.listdir(input_folder)

['.ipynb_checkpoints',
 'benchmark.ipynb',
 'city_name_eng.csv',
 'city_name_rus.csv',
 'coordinates.csv',
 'EDA.ipynb',
 'geocoding_and_ translation.ipynb',
 'hol.feather',
 'itc_en.feather',
 'items.csv',
 'item_categories.csv',
 'it_en.feather',
 'learn_error.tsv',
 'meta.tsv',
 'predict_future_sales.ipynb',
 'Programming_assignment_week_4 (1).ipynb',
 'references.txt',
 'sales_train.csv.gz',
 'sales_train_v2.csv',
 'sample_submission.csv',
 'sample_submission.csv.gz',
 'sample_submission1.csv',
 'shops.csv',
 'shops_city_longitude_latitude_en.csv',
 'sh_en.feather',
 'test.csv',
 'test.csv.gz',
 'time_left.tsv',
 'train']

In [13]:
shops = pd.read_csv('shops.csv')

In [14]:
from operator import itemgetter
shops['city'] = [i[0] if i[0][0]!='!' else i[0][1:] for i in shops.shop_name.apply(str.split)] # or shops.shop_name.apply(str.split).apply(itemgetter(0)).apply(str.strip,args='!')

## Translate City and Shop to english

In [15]:
from googletrans import Translator
translator = Translator()
translations_city = translator.translate(shops['city'].values.tolist(), dest='en')
translations_shop = translator.translate(shops['shop_name'].values.tolist(), dest='en')

In [16]:
shops['city_en'] = [a.text for a in translations_city]
shops['shop_name_en'] = [a.text for a in translations_shop]

In [17]:
shops.iloc[[9,12,55]] # shop_id 9, 12, 55 have no cities because they are online shops 

,shop_name,shop_id,city,city_en,shop_name_en
9,Выездная Торговля,9,Выездная,Traveling,Outbound Trade
12,Интернет-магазин ЧС,12,Интернет-магазин,Online store,Online shop Emergency
55,Цифровой склад 1С-Онлайн,55,Цифровой,Digital,Digital warehouse 1C-Online


## City/Shop longitude and latitude

In [23]:
import geocoder
shops['coordinates']=[geocoder.yandex(c).latlng for c in shops.city.values]
shops['lat']=shops.coordinates.drop([9,12,55]).apply(itemgetter(0)).apply(float)
shops['long']=shops.coordinates.drop([9,12,55]).apply(itemgetter(1)).apply(float)

ImportError: No module named builtins

In [21]:
shops

,shop_name,shop_id,city,city_en,shop_name_en
0,"!Якутск Орджоникидзе, 56 фран",0,Якутск,Yakutsk,"Yakutsk Ordzhonikidze, 56 francs"
1,"!Якутск ТЦ ""Центральный"" фран",1,Якутск,Yakutsk,"Yakutsk TC ""Central"" franc"
2,"Адыгея ТЦ ""Мега""",2,Адыгея,Adygea,"Adygea Shopping Center ""Mega"""
3,"Балашиха ТРК ""Октябрь-Киномир""",3,Балашиха,Balashiha,"Balashikha TRK ""October-Kinomir"""
4,"Волжский ТЦ ""Волга Молл""",4,Волжский,Volzhsky,"Volzhsky shopping center ""Volga Mall"""
5,"Вологда ТРЦ ""Мармелад""",5,Вологда,Vologda,"Vologda Shopping and Entertainment Center ""Mar..."
6,"Воронеж (Плехановская, 13)",6,Воронеж,Voronezh,"Voronezh (Plekhanovskaya, 13)"
7,"Воронеж ТРЦ ""Максимир""",7,Воронеж,Voronezh,"Voronezh TRC ""Maksimir"""
8,"Воронеж ТРЦ Сити-Парк ""Град""",8,Воронеж,Voronezh,"Voronezh TRC City-Park ""Grad"""
9,Выездная Торговля,9,Выездная,Traveling,Outbound Trade


In [ ]:
shops.to_csv('shops_city_longitude_latitude_en.csv',index=False)

## Map shop locations

In [ ]:
shops_summary=shops[~shops.lat.isna()].groupby('city').agg({'lat':np.mean,'long':np.mean,'shop_id':len}).reset_index()
shops_summary.rename({'shop_id':'city_cnt'},inplace=True,axis=1)
shops_summary

In [ ]:
%matplotlib inline
from mpl_toolkits.basemap import Basemap
import matplotlib.pyplot as plt
import builti%matplotlib inline
from mpl_toolkits.basemap import Basemap
import matplotlib.pyplot as plt
import builtinsns

plt.figure(figsize=(14,10))

map = Basemap(llcrnrlon=19.5,llcrnrlat=35,urcrnrlon=140,urcrnrlat=75)

map.drawcoastlines()
map.drawcountries()

x, y = map(shops_summary.long, shops_summary.lat)

map.scatter(x, y, marker='D',color='m')
for i in range(len(x)):
    plt.text(x[i],y[i],shops_summary.loc[i,'city'])


plt.show()

## Conclusion
A quick EDA using shops.csv yields these results:

- Variable shop_name seems to contain in it a city name for all except three shops (9, 12 and 55). A quick look at the distribution of the city locations suggests it can be used as a category for encoding shops.
- In addition, two shops may be duplicates, shop_id 10 and 11, with their names almost identical.
- Shops 39 and 40 may be two braches of the same store, differing only in last word of the name.